In [1]:
import pandas as pd

In [2]:
dfA = pd.read_csv("dsA.csv")
dfA.head()

,id,name,country,revenue_2022,profit,assets,market_value
0,0,berkshire hathaway,united states,doll_276.09 b,NaN,NaN,doll_741.48 b
1,1,icbc,china,doll_208.13 b,NaN,NaN,doll_214.43 b
2,"2,saudi arabian oil company (saudi aramco),sau...",NaN,NaN,NaN,NaN,NaN,NaN
3,3,jpmorgan chase,united states,doll_124.54 b,NaN,NaN,doll_374.45 b
4,4,china construction bank,china,doll_202.07 b,NaN,NaN,doll_181.32 b


In [21]:
df = pd.read_csv("predictions.csv")
df.head()

,match_score,ltable_id,rtable_id,ltable_name,ltable_country,ltable_revenue_2022,ltable_profit,ltable_assets,ltable_market_value,rtable_name,rtable_country,rtable_revenue_2022,rtable_profit,rtable_assets,rtable_market_value
0,0.784563,0,0,berkshire hathaway,united states,doll_276.09 b,NaN,NaN,doll_741.48 b,berkshire hathaway,united states,doll_276.09 b,doll_89.8 b,doll_958.78 b,doll_741.48 b
1,0.792979,9,9,toyota motor,japan,doll_281.75 b,NaN,NaN,doll_237.73 b,toyota motor,japan,doll_281.75 b,doll_28.15 b,doll_552.46 b,doll_237.73 b
2,0.781058,13,13,samsung group,south korea,doll_244.16 b,NaN,NaN,doll_367.26 b,samsung group,south korea,doll_244.16 b,doll_34.27 b,doll_358.88 b,doll_367.26 b
3,0.784673,18,18,verizon communications,united states,doll_134.35 b,NaN,NaN,doll_218.02 b,verizon communications,united states,doll_134.35 b,doll_21.52 b,doll_366.6 b,doll_218.02 b
4,0.786995,21,21,unitedhealth group,united states,doll_297.55 b,NaN,NaN,doll_490.15 b,unitedhealth group,united states,doll_297.55 b,doll_17.45 b,doll_221.24 b,doll_490.15 b


In [23]:
threshold = 0.5
dict_match = {}
df = df.reset_index()
for index, row in df.iterrows():
    if row["match_score"] > threshold:
        score = row["match_score"]
        l_id = str(row['ltable_id'])
        r_id = str(row['rtable_id'])
        if l_id not in dict_match.keys():
            dict_match[l_id] = set()
        dict_match[l_id].add(r_id) #((r_id, score))
        if r_id not in dict_match.keys():
            dict_match[r_id] = set()
        dict_match[r_id].add(l_id) #((l_id, score))

In [24]:
final_tuple_match = []
for key, values in dict_match.items():
    ok = False
    for sublist in final_tuple_match:
        if key in sublist:
            sublist.update(values)
            ok = True
    if not ok:
        new_set = set(values)
        final_tuple_match.append(new_set)

### Caso 1

In [26]:
def dict_list_to_dict_first_value(d):
    return {k: v[0] for k, v in d.items() if len(v) >= 1}

In [39]:
data_merged = []
for values in final_tuple_match:
    tmp_dict = {}
    for value in values:
        tmp_dict.update(dict_list_to_dict_first_value(dfA.loc[dfA['id'] == value].to_dict(orient="list")))
    data_merged.append(tmp_dict)

In [40]:
df = pd.DataFrame(data_merged)
df

,id,name,country,revenue_2022,profit,assets,market_value
0,0,berkshire hathaway,united states,doll_276.09 b,NaN,NaN,doll_741.48 b
1,9,toyota motor,japan,doll_281.75 b,NaN,NaN,doll_237.73 b
2,13,samsung group,south korea,doll_244.16 b,NaN,NaN,doll_367.26 b
3,18,verizon communications,united states,doll_134.35 b,NaN,NaN,doll_218.02 b
4,21,unitedhealth group,united states,doll_297.55 b,NaN,NaN,doll_490.15 b
...,...,...,...,...,...,...,...
526,993,yum! brands,canada,doll_6.58 b,NaN,NaN,doll_35.28 b
527,994,first quantum minerals,canada,doll_8.29 b,NaN,NaN,doll_19.43 b
528,995,aercap holdings,ireland,doll_4.52 b,NaN,NaN,doll_12.59 b
529,996,first horizon,united states,doll_3.15 b,NaN,NaN,doll_12.21 b


### Caso 2

In [35]:
def most_frequent(l):
    return max(set(l), key = l.count)

In [36]:
def merge_list_of_dict(df, values):
    keys = list(df.columns)
    dict_of_merge = {k: [] for k in keys}
    for value in values:
        tmp = dict_list_to_dict_first_value(df.loc[df['id'] == value].to_dict(orient="list"))
        for k, v in tmp.items():
            dict_of_merge[k].append(v)
    return {k: most_frequent(v) for k, v in dict_of_merge.items()}

In [37]:
res = []
for values in final_tuple_match:
    res.append(merge_list_of_dict(df, values))

In [38]:
df = pd.DataFrame(res)
df

,id,name,country,revenue_2022,profit,assets,market_value
0,0,berkshire hathaway,united states,doll_276.09 b,NaN,NaN,doll_741.48 b
1,9,toyota motor,japan,doll_281.75 b,NaN,NaN,doll_237.73 b
2,13,samsung group,south korea,doll_244.16 b,NaN,NaN,doll_367.26 b
3,18,verizon communications,united states,doll_134.35 b,NaN,NaN,doll_218.02 b
4,21,unitedhealth group,united states,doll_297.55 b,NaN,NaN,doll_490.15 b
...,...,...,...,...,...,...,...
526,993,yum! brands,canada,doll_6.58 b,NaN,NaN,doll_35.28 b
527,994,first quantum minerals,canada,doll_8.29 b,NaN,NaN,doll_19.43 b
528,995,aercap holdings,ireland,doll_4.52 b,NaN,NaN,doll_12.59 b
529,996,first horizon,united states,doll_3.15 b,NaN,NaN,doll_12.21 b
